<a href="https://colab.research.google.com/github/Akanksha-k4/ai-timetable/blob/main/Ai_Timetable_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools
!pip install streamlit
!pip install pandas
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 22.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import pandas as pd
from ortools.sat.python import cp_model

# Function to generate timetable
def generate_timetable(subjects, rooms, time_slots, days):
    model = cp_model.CpModel()

    # Decision variables: (Subject, Day, Time, Room)
    schedule = {}
    for index, row in subjects.iterrows():
        subject = row['Subject']
        for day in days:
            for time in time_slots:
                for room in rooms:
                    schedule[(subject, day, time, room)] = model.NewBoolVar(f"{subject}_{day}_{time}_{room}")

    # Debugging: Check if all variables are created
    print(f"✅ Created {len(schedule)} scheduling variables")

    # Constraint 1: Each subject must be scheduled at least once
    for subject in subjects['Subject']:
        model.Add(sum(schedule[(subject, day, time, room)] for day in days for time in time_slots for room in rooms) >= 1)

    # Constraint 2: No two subjects in the same room at the same time
    for day in days:
        for time in time_slots:
            for room in rooms:
                model.Add(sum(schedule[(subject, day, time, room)] for subject in subjects['Subject']) <= len(rooms))

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # If solution found, return timetable
    if status in (cp_model.FEASIBLE, cp_model.OPTIMAL):
        timetable = {day: [] for day in days}
        for (subject, day, time, room), var in schedule.items():
            if solver.Value(var) == 1:
                timetable[day].append((subject, time, room))

        # Convert timetable to DataFrame
        max_classes = max(len(timetable[day]) for day in days)
        df_data = {day: timetable[day] + [(" ", " ", " ")] * (max_classes - len(timetable[day])) for day in days}
        timetable_df = pd.DataFrame(df_data)
        timetable_df.columns = [f"📅 {day}" for day in days]

        return timetable_df
    else:
        print("⚠️ No feasible solution found! Check constraints.")
        return None

# Streamlit UI
st.title("📅 AI-Based College Timetable Generator")

# Upload files
uploaded_files = st.file_uploader("Upload CSV Files (Subjects, Rooms, Time Slots)", accept_multiple_files=True, type=["csv"])

if uploaded_files:
    dataframes = {}
    for uploaded_file in uploaded_files:
        df = pd.read_csv(uploaded_file)
        dataframes[uploaded_file.name] = df
        st.write(f"### {uploaded_file.name}")
        st.dataframe(df)

    required_files = ["Subjects.csv", "Rooms.csv", "TimeSlots.csv"]
    uploaded_filenames = [file.name for file in uploaded_files]
    missing_files = [file for file in required_files if file not in uploaded_filenames]

    if missing_files:
        st.warning(f"⚠️ Missing required files: {', '.join(missing_files)}")
    else:
        st.success("✅ All required files uploaded! Ready to generate timetable.")

        # Extract data
        subjects_df = dataframes["Subjects.csv"]
        rooms_df = dataframes["Rooms.csv"]
        time_slots_df = dataframes["TimeSlots.csv"]

        # Check if column names are correct
        if 'Room Number' not in rooms_df.columns or 'Time Slot' not in time_slots_df.columns:
            st.error("⚠️ CSV files do not contain required columns! Check column names.")
        else:
            rooms = rooms_df['Room Number'].tolist()
            time_slots = time_slots_df['Time Slot'].tolist()
            days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

            if st.button("Generate Timetable"):
                timetable_df = generate_timetable(subjects_df, rooms, time_slots, days)
                if timetable_df is not None:
                    st.write("### 📌 Generated Timetable (Day-wise):")
                    st.dataframe(timetable_df)
                else:
                    st.error("⚠️ No feasible timetable found. Please check time slots, rooms, and constraints.")


2025-03-05 06:52:43.735 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-05 06:52:43.827 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-05 06:52:43.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-05 06:52:43.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-05 06:52:43.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-05 06:52:43.831 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-05 06:52:43.832 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-05 06:52:43.833 Thread 'MainThread': mi